Задание 6. Предварительная обработка данных

Инструкция:

Шаг 1.  Изучите материалы лекционных и практических занятий по темам раздела 4.

Шаг 2. Выполните предварительную обработку данных (очистку данных) предложенного датасета.  (Алгоритм выполнения представлен в практическом занятии по теме 4.3).

2.1 Загрузите исходные данные;

2.2 Изучите типы данных, наличие пропущенных значений, дублирующихся записей (признаков);

2.3 Изучите описательные статистики признаков. Выделите признаки, требующие, по Вашему мнению, предварительной обработки;

2.4 Для выделенных признаков выполните предварительную обработку (по необходимости): корректировка типов данных; работа с дублями, поиск и обработка аномальных значений (по значениям 1-99); обработка пропущенных значений.    

Шаг 3. Опубликуйте файл расширения ipynb на платформе Odin.

In [52]:
import pandas as pd
import re
re_cslsplit = re.compile(r'\s*,\s*')

In [31]:
df_cyr = pd.read_csv('./dataset_telecom+(2).csv')
df_cyr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4492 entries, 0 to 4491
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Возраст                                4492 non-null   int64  
 1   Среднемесячный расход                  4468 non-null   float64
 2   Средняя продолжительность разговоров   4475 non-null   float64
 3   Звонков днем за месяц                  4472 non-null   float64
 4   Звонков вечером за месяц               4489 non-null   float64
 5   Звонков ночью за месяц                 4492 non-null   object 
 6   Звонки в другие города                 4492 non-null   object 
 7   Звонки в другие страны                 4492 non-null   int64  
 8   Доля звонков на стационарные телефоны  4492 non-null   object 
 9   Количество SMS за месяц                4492 non-null   object 
 10  Дата подключения тарифа                4492 non-null   object 
dtypes: f

In [47]:
# корректировка заголовков
header_translate_dict = {'Возраст': 'age', 'Среднемесячный расход': 'avg_bill',
                        'Средняя продолжительность разговоров': 'avg_duration',
                        'Звонков днем за месяц': 'calls_day', 'Звонков вечером за месяц': 'calls_evening',
                        'Звонков ночью за месяц': 'calls_night', 'Звонки в другие города': 'calls_intercity',
                        'Звонки в другие страны': 'calls_international', 
                        'Доля звонков на стационарные телефоны': 'fraction_stationary',
                        'Количество SMS за месяц': 'num_sms', 'Дата подключения тарифа': 'date_activated'}
df = df_cyr.copy()
df.columns = df_cyr.columns.map(header_translate_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4492 entries, 0 to 4491
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  4492 non-null   int64  
 1   avg_bill             4468 non-null   float64
 2   avg_duration         4475 non-null   float64
 3   calls_day            4472 non-null   float64
 4   calls_evening        4489 non-null   float64
 5   calls_night          4492 non-null   object 
 6   calls_intercity      4492 non-null   object 
 7   calls_international  4492 non-null   int64  
 8   fraction_stationary  4492 non-null   object 
 9   num_sms              4492 non-null   object 
 10  date_activated       4492 non-null   object 
dtypes: float64(4), int64(2), object(5)
memory usage: 386.2+ KB


In [51]:
# поиск и обработка пропущенных значений.
nan_index = df.isna().agg(any, axis=1)   # в строке есть хоть один NaN
#len(df[nan_index])  # 62 < 15% -> допустимо удалить
# df = df[~nan_index] # 
df_nan = df.dropna(inplace=False)
df_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4430 entries, 1 to 4491
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  4430 non-null   int64  
 1   avg_bill             4430 non-null   float64
 2   avg_duration         4430 non-null   float64
 3   calls_day            4430 non-null   float64
 4   calls_evening        4430 non-null   float64
 5   calls_night          4430 non-null   object 
 6   calls_intercity      4430 non-null   object 
 7   calls_international  4430 non-null   int64  
 8   fraction_stationary  4430 non-null   object 
 9   num_sms              4430 non-null   object 
 10  date_activated       4430 non-null   object 
dtypes: float64(4), int64(2), object(5)
memory usage: 415.3+ KB


In [75]:
# корректирвока типов данных;
int_cols = 'age, calls_day, calls_evening, calls_night, calls_intercity, calls_international, num_sms'
float_cols = 'avg_bill, avg_duration, fraction_stationary'

typedict = {'date_activated':'datetime64[ns]'}
typedict.update({col: 'int64' for col in re_cslsplit.split(int_cols)})
typedict.update({col: 'float' for col in re_cslsplit.split(float_cols)})
#set(typedict.keys()) == set(df_nan.columns)

# # df_type = df_nan.astype(typedict)  # где-то '7' вместо числа!!
# # df_type.info()
# quoted_idx = df_nan.applymap(lambda x: False if re.fullmatch(r"['].*[']", str(x)) is None else True).agg(any, axis=1)
# df_nan[quoted_idx]  # проблемные кавычки
df_nan_fixed = df_nan.applymap(lambda x: x if re.fullmatch(r"['].*[']", str(x)) is None 
                               else re.fullmatch(r"['](.*)[']", str(x))[1])
df_type = df_nan_fixed.astype(typedict)
df_type.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4430 entries, 1 to 4491
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   age                  4430 non-null   int64         
 1   avg_bill             4430 non-null   float64       
 2   avg_duration         4430 non-null   float64       
 3   calls_day            4430 non-null   int64         
 4   calls_evening        4430 non-null   int64         
 5   calls_night          4430 non-null   int64         
 6   calls_intercity      4430 non-null   int64         
 7   calls_international  4430 non-null   int64         
 8   fraction_stationary  4430 non-null   float64       
 9   num_sms              4430 non-null   int64         
 10  date_activated       4430 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(7)
memory usage: 544.4 KB


In [79]:
# поиск и обработка дублирующихся записей;
df_type.duplicated().sum() # == 0 -> действие не требуется

0

In [80]:
# поиск и обработка аномальных наблюдений;
df_type.describe(percentiles=[.01, .05, .5, .95, .99])

,age,avg_bill,avg_duration,calls_day,calls_evening,calls_night,calls_intercity,calls_international,fraction_stationary,num_sms
count,4430.00000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000
mean,41.88307,506.384546,4.226309,63.907223,70.293905,6.261400,8.682167,0.395485,10.330474,21.160497
std,13.07620,648.257234,3.008247,63.004533,41.222571,14.674194,13.499074,1.177910,9.246434,27.764674
min,19.00000,3.180000,0.100000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1%,19.00000,7.778700,0.400000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5%,20.00000,16.800000,1.100000,5.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,43.00000,315.155000,3.300000,53.000000,71.000000,1.000000,2.000000,0.000000,8.000000,6.000000
95%,63.00000,1620.494000,9.700000,133.550000,142.000000,35.000000,39.000000,2.000000,27.000000,78.000000
99%,69.00000,3546.472300,14.542000,392.840000,156.000000,83.710000,46.710000,6.000000,36.710000,120.710000
max,70.00000,5142.760000,20.000000,500.000000,160.000000,110.000000,88.000000,12.000000,60.000000,179.000000


In [175]:
# "ящик с усами"
df_box1 = pd.DataFrame([df_type.quantile(0.25), df_type.quantile(0.75)], index=['q25', 'q75'])
iqr = df_box1.loc['q75'] -df_box1.loc['q25']
low = df_box1.loc['q25'] -1.5*iqr
high = df_box1.loc['q75'] +1.5*iqr
df_box2 = pd.concat([df_box1, pd.DataFrame([iqr, low, high], 'iqr,low,high'.split(','))])

anom_tbl = []
for col in df_box2:
    high, low = df_box2.loc[['high', 'low'], col]
#     print(col, high, low)
    anomalies = df_type[col].between(low, high, 'both')
    anom_tbl.append(anomalies)
anom_tbl = pd.DataFrame(anom_tbl).T
df_box2 = pd.concat([df_box2, pd.DataFrame(anom_tbl.agg(sum), columns=['cnt_valid']).T])
display(df_box2)

# признаки по которым будут удаляться аномальные значения (в порядке перечисления категорий)
# normalize_by_cols = ['calls_international']
# normalize_by_cols = ['num_sms']
normalize_by_cols = ['avg_bill', 'avg_duration', 'calls_day', 'calls_evening']
# normalize_by_cols = ['calls_night']

df_anom = df_type[anom_tbl[normalize_by_cols].agg(all, axis=1)]
print('Отфильтровано %d аномальных значений. Потери сосавили %.1f%% записей\n' % 
      (len(df_type)-len(df_anom),(len(df_type)-len(df_anom))/len(df_type)*100) )
df_anom.info()
df_anom.describe(percentiles=[.01, .05, .5, .95, .99])


,age,avg_bill,avg_duration,calls_day,calls_evening,calls_night,calls_intercity,calls_international,fraction_stationary,num_sms
q25,31.0,152.88,2.00,37.0,41.250,0.0,0.0,0.0,3.0,3.0
q75,52.0,599.96,5.90,68.0,98.000,5.0,12.0,0.0,16.0,32.0
iqr,21.0,447.08,3.90,31.0,56.750,5.0,12.0,0.0,13.0,29.0
low,-0.5,-517.74,-3.85,-9.5,-43.875,-7.5,-18.0,0.0,-16.5,-40.5
high,83.5,1270.58,11.75,114.5,183.125,12.5,30.0,0.0,35.5,75.5
cnt_valid,4430.0,4106.00,4306.00,4160.0,4430.000,3917.0,3942.0,3620.0,4381.0,4196.0


Отфильтровано 371 аномальных значений. Потери сосавили 8.4% записей

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4059 entries, 1 to 4491
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   age                  4059 non-null   int64         
 1   avg_bill             4059 non-null   float64       
 2   avg_duration         4059 non-null   float64       
 3   calls_day            4059 non-null   int64         
 4   calls_evening        4059 non-null   int64         
 5   calls_night          4059 non-null   int64         
 6   calls_intercity      4059 non-null   int64         
 7   calls_international  4059 non-null   int64         
 8   fraction_stationary  4059 non-null   float64       
 9   num_sms              4059 non-null   int64         
 10  date_activated       4059 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(7)
memory usage: 380.5 KB


,age,avg_bill,avg_duration,calls_day,calls_evening,calls_night,calls_intercity,calls_international,fraction_stationary,num_sms
count,4059.000000,4059.000000,4059.000000,4059.000000,4059.000000,4059.000000,4059.000000,4059.000000,4059.000000,4059.000000
mean,42.050505,353.055127,3.673602,49.994334,66.790589,4.198078,7.473516,0.264351,9.586105,19.873368
std,13.157336,283.229844,2.263913,25.417407,40.497498,11.416968,12.180363,0.656197,8.151454,25.573544
min,19.000000,3.180000,0.100000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1%,19.000000,7.509600,0.300000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5%,20.000000,15.797000,1.100000,5.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,43.000000,287.260000,3.000000,51.000000,67.000000,0.000000,0.000000,0.000000,8.000000,5.000000
95%,63.000000,942.827000,8.200000,99.000000,140.000000,21.000000,36.000000,2.000000,26.000000,70.000000
99%,69.000000,1175.146600,9.600000,111.000000,156.000000,67.000000,44.000000,3.000000,29.000000,114.420000
max,70.000000,1270.080000,10.200000,114.000000,160.000000,108.000000,49.000000,3.000000,30.000000,172.000000


 __"Аномальные" категории (по-видимому объясняются независимыми переменными) это:__
1. пользователи международной связи
2. писатели SMS
3. "Киты" -- те, кто тратит значительно больше среднего, совершает много звонков днём/вечером
4. "Совы" -- те, кто звонит ночью